In [1]:
import torch
torch.cuda.get_device_name(0)

'GeForce 940M'

In [2]:
#DFANet模型搭建
import time
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo

class FPA(nn.Module):
    def __init__(self, channels=2048):
        """
        Feature Pyramid Attention
        :type channels: int
        """
        super(FPA, self).__init__()
        channels_mid = int(channels/4)

        self.channels_cond = channels

        # Master branch
        self.conv_master = nn.Conv2d(self.channels_cond, channels, kernel_size=1, bias=False)
        self.bn_master = nn.BatchNorm2d(channels)

        # Global pooling branch
        self.conv_gpb = nn.Conv2d(self.channels_cond, channels, kernel_size=1, bias=False)
        self.bn_gpb = nn.BatchNorm2d(channels)

        # C333 because of the shape of last feature maps is (16, 16).
        self.conv7x7_1 = nn.Conv2d(self.channels_cond, channels_mid, kernel_size=(7, 7), stride=2, padding=3, bias=False)
        self.bn1_1 = nn.BatchNorm2d(channels_mid)
        self.conv5x5_1 = nn.Conv2d(channels_mid, channels_mid, kernel_size=(5, 5), stride=2, padding=2, bias=False)
        self.bn2_1 = nn.BatchNorm2d(channels_mid)
        self.conv3x3_1 = nn.Conv2d(channels_mid, channels_mid, kernel_size=(3, 3), stride=2, padding=1, bias=False)
        self.bn3_1 = nn.BatchNorm2d(channels_mid)

        self.conv7x7_2 = nn.Conv2d(channels_mid, channels_mid, kernel_size=(7, 7), stride=1, padding=3, bias=False)
        self.bn1_2 = nn.BatchNorm2d(channels_mid)
        self.conv5x5_2 = nn.Conv2d(channels_mid, channels_mid, kernel_size=(5, 5), stride=1, padding=2, bias=False)
        self.bn2_2 = nn.BatchNorm2d(channels_mid)
        self.conv3x3_2 = nn.Conv2d(channels_mid, channels_mid, kernel_size=(3, 3), stride=1, padding=1, bias=False)
        self.bn3_2 = nn.BatchNorm2d(channels_mid)

        # Convolution Upsample
        self.conv_upsample_3 = nn.ConvTranspose2d(channels_mid, channels_mid, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn_upsample_3 = nn.BatchNorm2d(channels_mid)

        self.conv_upsample_2 = nn.ConvTranspose2d(channels_mid, channels_mid, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn_upsample_2 = nn.BatchNorm2d(channels_mid)

        self.conv_upsample_1 = nn.ConvTranspose2d(channels_mid, channels, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn_upsample_1 = nn.BatchNorm2d(channels)

        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        """
        :param x: Shape: [b, 2048, h, w]
        :return: out: Feature maps. Shape: [b, 2048, h, w]
        """
        # Master branch
        x_master = self.conv_master(x)
        x_master = self.bn_master(x_master)

        # Global pooling branch
        x_gpb = nn.AvgPool2d(x.shape[2:])(x).view(x.shape[0], self.channels_cond, 1, 1)
        x_gpb = self.conv_gpb(x_gpb)
        x_gpb = self.bn_gpb(x_gpb)

        # Branch 1
        #print('x:',x.shape)
        x1_1 = self.conv7x7_1(x)
        #print('after con7x7 x:',x1_1.shape)
        x1_1 = self.bn1_1(x1_1)
        x1_1 = self.relu(x1_1)
        x1_2 = self.conv7x7_2(x1_1)
        x1_2 = self.bn1_2(x1_2)

        # Branch 2
        x2_1 = self.conv5x5_1(x1_1)
        x2_1 = self.bn2_1(x2_1)
        x2_1 = self.relu(x2_1)
        x2_2 = self.conv5x5_2(x2_1)
        x2_2 = self.bn2_2(x2_2)

        # Branch 3
        x3_1 = self.conv3x3_1(x2_1)
        x3_1 = self.bn3_1(x3_1)
        x3_1 = self.relu(x3_1)
        x3_2 = self.conv3x3_2(x3_1)
        x3_2 = self.bn3_2(x3_2)

        # Merge branch 1 and 2
        x3_upsample = self.relu(self.bn_upsample_3(self.conv_upsample_3(x3_2)))
        
        diffY = torch.tensor([x3_upsample.size()[2] - x2_2.size()[2]])
        diffX = torch.tensor([x3_upsample.size()[3] - x2_2.size()[3]])
        x2_2 = F.pad(x2_2, [diffX // 2, diffX - diffX // 2, 
                          diffY // 2, diffY - diffY // 2])
        
        x2_merge = self.relu(x2_2 + x3_upsample)
        
        x2_upsample = self.relu(self.bn_upsample_2(self.conv_upsample_2(x2_merge)))

        diffY = torch.tensor([x2_upsample.size()[2] - x1_2.size()[2]])
        diffX = torch.tensor([x2_upsample.size()[3] - x1_2.size()[3]])
 
        x1_2 = F.pad(x1_2, [diffX // 2, diffX - diffX // 2, 
                          diffY // 2, diffY - diffY // 2])
    
        #print('x1_2 + x2_upsample:', x1_2.shape, x2_upsample.shape)
        x1_merge = self.relu(x1_2 + x2_upsample)
        
        x1_upsample = self.relu(self.bn_upsample_1(self.conv_upsample_1(x1_merge)))
        
        diffY = torch.tensor([x1_upsample.size()[2] - x_master.size()[2]])
        diffX = torch.tensor([x1_upsample.size()[3] - x_master.size()[3]])
 
        x_master = F.pad(x_master, [diffX // 2, diffX - diffX // 2, 
                          diffY // 2, diffY - diffY // 2])

        x_master = x_master * x1_upsample

        out = self.relu(x_master + x_gpb)

        return out

#可分离卷积
class SeparableConv2d(nn.Module):
    def __init__(self, inputChannel, outputChannel, kernel_size=3, stride=1, padding=1, dilation=1, bias=True):
        super(SeparableConv2d, self).__init__()
        self.conv1 = nn.Conv2d(inputChannel, inputChannel, kernel_size, stride, padding, dilation,
                               groups=inputChannel, bias=bias)
        self.pointwise = nn.Conv2d(inputChannel, outputChannel, 1, 1, 0, 1, 1, bias=bias)
    def forward(self, x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


# encoder block
class Block(nn.Module):
    """
    Base block for XceptionA and DFANet.
    inputChannel: channels of inputs of the base block.
    outputChannel:channnels of outputs of the base block.
    stride: stride
    BatchNorm:
    """
    def __init__(self, inputChannel, outputChannel,stride=1,BatchNorm=nn.BatchNorm2d):
        super(Block, self).__init__()

        self.conv1=nn.Sequential(SeparableConv2d(inputChannel,outputChannel//4,stride=stride,),
                                BatchNorm(outputChannel//4),
                                nn.ReLU())
        self.conv2=nn.Sequential(SeparableConv2d(outputChannel//4,outputChannel//4),
                                BatchNorm(outputChannel//4),
                                nn.ReLU())
        self.conv3=nn.Sequential(SeparableConv2d(outputChannel//4,outputChannel),
                                BatchNorm(outputChannel),
                                nn.ReLU())
        self.projection=nn.Conv2d(inputChannel,outputChannel,1,stride=stride,bias=False)
        
       
    def forward(self, x):
        out=self.conv1(x)
        out=self.conv2(out)
        out=self.conv3(out)
        identity=self.projection(x)
        return out+identity


class enc(nn.Module):
    """
    encoders:
    in_channels:The channels of input feature maps
    out_channnel:the channels of outputs of this enc.
    """
    def __init__(self,in_channels,out_channels,stride=2,num_repeat=3):
        super(enc, self).__init__()
        stacks=[Block(in_channels,out_channels,stride=2)]
        for x in range(num_repeat-1):
            stacks.append(Block(out_channels,out_channels))
        self.build=nn.Sequential(*stacks)
        # self.block1=Block(in_channels,out_channels,stride=2)
        # self.block2=Block(out_channels,out_channels)
        # self.block3=Block(out_channels,out_channels)
    def forward(self, x):
        x=self.build(x)
        # x=self.block2(x)
        # x=self.block3(x)
        return x

class Attention(nn.Module):
    """
    self attention model.
    """
    def __init__(self,in_channels,out_channels):
        super(Attention,self).__init__()
        self.avg_pool=nn.AdaptiveAvgPool2d(1)
        self.fc=nn.Sequential(
            nn.Linear(in_channels,1000,bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(1000, out_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self,x):
        b,c,_,_=x.size()
        y=self.avg_pool(x).view(b,c)
        y=self.fc(y).view(b,c,1,1)
        return x*y.expand_as(x)

        
class SubBranch(nn.Module):
    """
    channel_cfg: channels of the inputs of enc stage
    branch_index: 0,1,2,
    """
    def __init__(self,channel_cfg,branch_index):
        super(SubBranch,self).__init__()
        self.enc2=enc(channel_cfg[0],48,num_repeat=3)
        self.enc3=enc(channel_cfg[1],96,num_repeat=6)
        self.enc4=enc(channel_cfg[2],192,num_repeat=3)
        self.atten=Attention(192,192)
        self.atten_fpa=FPA(192)
        self.branch_index=branch_index
    
    def forward(self,x0,*args):
        out0=self.enc2(x0)
        if self.branch_index in [1,2]:
            out1=self.enc3(torch.cat([out0,args[0]],1))
            out2=self.enc4(torch.cat([out1,args[1]],1))
        else:
            out1=self.enc3(out0)
            out2=self.enc4(out1)
        if self.branch_index in [0,1,2]:
          out3=self.atten(out2)
        else:
          out3=self.atten_fpa(out2)
        return [out0,out1,out2,out3]    

class XceptionA(nn.Module):
    """
    channel_cfg:channels of inputs of enc block.
    n_classes:
    """
    def __init__(self,channel_cfg,n_classes):
        super(XceptionA, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3,stride=2,padding=1,bias=False),
                                nn.BatchNorm2d(num_features=8),
                                nn.ReLU())
        self.branch=SubBranch(channel_cfg,branch_index=0)
        self.avg_pool=nn.AdaptiveAvgPool2d(1)
    
        self.classifier=nn.Sequential(nn.Linear(192,1000),
                                      nn.ReLU(inplace=True),
                                      nn.Linear(1000,n_classes))
        for m in self.modules():
            weight_init(m)

    def forward(self,x):
        b,c,_,_=x.size()
        x=self.conv1(x)
        _,_,_,x=self.branch(x)
        x=self.avg_pool(x).view(b,-1)
        x=self.classifier(x)
        return x

class DFA_Encoder(nn.Module):
    """
    Encoder of DFANet.
    """
    def __init__(self,channel_cfg):
        super(DFA_Encoder,self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3,stride=2,padding=1,bias=False),
                                nn.BatchNorm2d(num_features=8),
                                nn.ReLU())
        self.branch0=SubBranch(channel_cfg[0],branch_index=0)
        self.branch1=SubBranch(channel_cfg[1],branch_index=1)
        self.branch2=SubBranch(channel_cfg[2],branch_index=2)

    def forward(self,x):

        x=self.conv1(x)
        x0,x1,x2,x5=self.branch0(x)
        bch0 = time.process_time()
        x3=F.interpolate(x5,x0.size()[2:],mode='bilinear',align_corners=True)
        x1,x2,x3,x6=self.branch1(torch.cat([x0,x3],1),x1,x2)
        bch1 = time.process_time()
        x4=F.interpolate(x6,x1.size()[2:],mode='bilinear',align_corners=True)
        x2,x3,x4,x7=self.branch2(torch.cat([x1,x4],1),x2,x3)
        bch2 = time.process_time()
        return [x0,x1,x2,x5,x6,x7]


class DFA_Decoder(nn.Module):
    """
        Decoder of DFANet.
    """
    def __init__(self,decode_channels,n_classes):
        super(DFA_Decoder,self).__init__()


        self.conv0=nn.Sequential(nn.Conv2d(in_channels=48,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
        self.conv1=nn.Sequential(nn.Conv2d(in_channels=48,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
        self.conv2=nn.Sequential(nn.Conv2d(in_channels=48,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
        self.conv3=nn.Sequential(nn.Conv2d(in_channels=192,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
        self.conv4=nn.Sequential(nn.Conv2d(in_channels=192,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
        self.conv5=nn.Sequential(nn.Conv2d(in_channels=192,out_channels=decode_channels,kernel_size=1,bias=False),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))

        self.conv_add1=nn.Sequential(nn.Conv2d(in_channels=decode_channels,out_channels=decode_channels,kernel_size=3,padding=1),
                                 nn.BatchNorm2d(decode_channels),
                                 nn.ReLU(inplace=True))
                                

        self.conv_cls=nn.Conv2d(in_channels=decode_channels,out_channels=n_classes,kernel_size=3,padding=1,bias=False)

    def forward(self,x0,x1,x2,x3,x4,x5):

        x0=self.conv0(x0)
        x1=F.interpolate(self.conv1(x1),x0.size()[2:],mode='bilinear',align_corners=True)
        x2=F.interpolate(self.conv2(x2),x0.size()[2:],mode='bilinear',align_corners=True)
        x3=F.interpolate(self.conv3(x3),x0.size()[2:],mode='bilinear',align_corners=True)
        x4=F.interpolate(self.conv5(x4),x0.size()[2:],mode='bilinear',align_corners=True)
        x5=F.interpolate(self.conv5(x5),x0.size()[2:],mode='bilinear',align_corners=True)

        x_shallow=self.conv_add1(x0+x1+x2)

        x=self.conv_cls(x_shallow+x3+x4+x5)
        x=F.interpolate(x,scale_factor=4,mode='bilinear',align_corners=True)
        return x

class DFANet(nn.Module):
    def __init__(self,channel_cfg,decoder_channel,n_classes):
        super(DFANet,self).__init__()
        self.n_classes = n_classes
        self.encoder=DFA_Encoder(channel_cfg)
        self.decoder=DFA_Decoder(decoder_channel,n_classes)
        weight_init(self.encoder)
        weight_init(self.decoder)
    def forward(self,x):
        x0,x1,x2,x3,x4,x5=self.encoder(x)
        x=self.decoder(x0,x1,x2,x3,x4,x5)
        return x

def weight_init(module):
    #print('initialize  ',module._get_name())
    for n,m in module.named_children():
        if isinstance(m,nn.Conv2d):
            nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m,(nn.BatchNorm2d,nn.InstanceNorm2d)):
            nn.init.ones_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m,nn.Linear):
            nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m,(nn.Sequential,
                            SubBranch,
                            enc,Block,
                            SeparableConv2d,
                            Attention,
                            DFA_Encoder,
                            DFA_Decoder)):
            weight_init(m)
        elif isinstance(m,(nn.ReLU,nn.ReLU,nn.ReLU6)):
            pass
        else:
            pass

In [3]:
import time
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
ch_cfg=[[8,48,96],
    [240,144,288],
    [240,144,288]]


input = torch.randn(4, 3, 360, 480)
input=input.to(device)


#decoder test
#input=input.to(device)
net=DFANet(ch_cfg,64,8)
net=net.to(device)
net(input)
start=time.time()
outputs=net(input)
end=time.time()

print(outputs.size())
print("inference time",end-start)
print("test DFANet sucessfully...")

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 276480 bytes. Buy new RAM!


In [0]:
#UNeth模型各模块搭建
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

#UNet模型构建
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 256)
        self.up1 = Up(512, 128, bilinear)
        self.up2 = Up(256, 64, bilinear)
        self.up3 = Up(128, 32, bilinear)
        self.up4 = Up(64, 32, bilinear)
        self.outc = OutConv(32, self.n_classes)
        if self.n_classes == 1:
            self.out  = torch.sigmoid
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        if self.n_classes == 1:
            logits = self.out(logits)
        return logits

net = UNet(n_channels=3, n_classes=8)

In [6]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 30 18:50:07 2019

@author: Mryan
"""

class AverageMeter:
    def __init__(self):
        self.value = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.value = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val):
        raise NotImplementedError()

    def update_n(self, val, n):
        raise NotImplementedError()

    @property
    def val(self):
        return self.avg


class FPSMeter():
    """
    Class to measure frame per second in our networks
    """

    def __init__(self):
        self.frame_per_second = 0.0
        self.f_in_milliseconds = 0.0

        self.frame_count = 0
        self.milliseconds = 0.0

    def reset(self):
        self.frame_per_second = 0.0
        self.f_in_milliseconds = 0.0

        self.frame_count = 0

    def update(self, seconds):
        self.milliseconds += seconds * 1000
        self.frame_count += 1

        self.frame_per_second = self.frame_count / (self.milliseconds / 1000.0)
        self.f_in_milliseconds = self.milliseconds / self.frame_count

    @property
    def mspf(self):
        return self.f_in_milliseconds

    @property
    def fps(self):
        return self.frame_per_second

    def print_statistics(self):
        print("""
Statistics of the FPSMeter
Frame per second: {:.2f} fps
Milliseconds per frame: {:.2f} ms in one frame
These statistics are calculated based on
{:d} Frames and the whole taken time is {:.4f} Seconds
        """.format(self.frame_per_second, self.f_in_milliseconds, self.frame_count, self.milliseconds / 1000.0))


def main_test_fps():
    fpsfps = FPSMeter()
    fpsfps.update(0.025)
    fpsfps.update(0.025)
    fpsfps.update(0.025)
    fpsfps.update(0.025)
    fpsfps.print_statistics()


if __name__ == '__main__':
    main_test_fps()


Statistics of the FPSMeter
Frame per second: 40.00 fps
Milliseconds per frame: 25.00 ms in one frame
These statistics are calculated based on
4 Frames and the whole taken time is 0.1000 Seconds
        


In [8]:
import os
import torch
import time

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def test():

    #model = NetWork(19).cuda()
    #model = net.cuda()
    model =net.to(device)
    model.eval()
    
    fps_meter = FPSMeter()

    for i in range(0,1000):

        input = torch.randn(1,3,360,480).cuda()

        with torch.no_grad():
            
            torch.cuda.synchronize()
            torch.cuda.synchronize()
            
            start = time.perf_counter()
            
            output = model(input)
            
            torch.cuda.synchronize()
            
            end = time.perf_counter()
            fps_meter.update(end-start)

    fps_meter.print_statistics()

if __name__ == "__main__":
    
    test()

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same